In [46]:
import pickle
import matplotlib.pyplot as plt # creating visualizations
import numpy as np # basic math and random numbers
import torch # package for building functions with learnable parameters
import torch.nn as nn # prebuilt functions specific to neural networks
from torch.autograd import Variable # storing data while learning
#from disp import ADE,FDE,prediction_displacement,ADE_double_coordinates,FDE_double_coordinates,prediction_displacement_double


In [48]:
import numpy as np
import pandas as pd
# Predict error
# Average Displacement error

def ADE(pred,truth): 
    counter=0
    sum=0
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        for j in range (half):

            a = np.array((pred[i][j] , pred[i][half+j]))
            b = np.array((truth.iloc[i][j] , truth.iloc[i][half+j]))

            dist = np.linalg.norm(a-b)
            sum+=dist
            counter+=1
            #print("Distance between",a," and ",b," is: ",dist)

    return (sum/counter)
def prediction_displacement_double(pred): 
          
        sum=0
        counter=0        
        pred=np.array(pred)
        last_index= (len(pred[0])-1)
        
        for i in range(len(pred)):
                
                    a = np.array((pred[i][0][0] , pred[i][0][1]))
                    b = np.array((pred[i][last_index][0] , pred[i][last_index][1]))

                    dist = np.linalg.norm(a-b)
                    sum+=dist
                    counter+=1
        return (sum/counter)

def prediction_displacement(pred): 
    counter=0
    sum=0
    pred=np.array(pred)
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        last=(len(pred[i]) - 1)

        a = np.array((pred[i][0] , pred[i][half]))
        b = np.array((pred[i][half-1] , pred[i][last]))

        dist = np.linalg.norm(a-b)
        sum+=dist
        counter+=1
        #print("FDE Distance between",a," and ",b," is: ",dist)
            
    return (sum/counter)
def FDE(pred,truth): 
    counter=0
    sum=0
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        last=(len(pred[i]) - 1)

        a = np.array((pred[i][half-1] , pred[i][last]))
        b = np.array((truth.iloc[i][half-1] , truth.iloc[i][last]))

        dist = np.linalg.norm(a-b)
        sum+=dist
        counter+=1
        #print("FDE Distance between",a," and ",b," is: ",dist)
            
    return (sum/counter)


def FDE_double_coordinates(pred,truth):
          
        sum=0
        counter=0
        
        pred=np.array(pred)
        truth=np.array(truth)
        last_index= (len(pred[0])-1)

        for i in range(len(pred)):
                
            a = np.array((pred[i][last_index][0] , pred[i][last_index][1]))
            b = np.array((truth[i][last_index][0] , truth[i][last_index][1]))

            dist = np.linalg.norm(a-b)
            sum+=dist
            counter+=1

                # for j in range (len(pred[i])):
                #    pred_x.append(pred[i][j][0])  
                #    pred_y.append(pred[i][j][1]) 

                #    truth_x.append(truth[i][j][0])  
                #    truth_y.append(truth[i][j][1])
        return (sum/counter)
def ADE_double_coordinates(pred,truth):
          
        sum=0
        counter=0
        
        pred=np.array(pred)
        truth=np.array(truth)

        for i in range(len(pred)):
                
                for j in range (len(pred[i])):

                    a = np.array((pred[i][j][0] , pred[i][j][1]))
                    b = np.array((truth[i][j][0] , truth[i][j][1]))

                    dist = np.linalg.norm(a-b)
                    sum+=dist
                    counter+=1
        return (sum/counter)

In [2]:
def load_data():
    with open('train_input.pickle', 'rb') as data:
     train_input = pickle.load(data)
    with open('validate_input.pickle', 'rb') as data:
         validate_input= pickle.load(data)
    with open('test_input.pickle', 'rb') as data:
         test_input = pickle.load(data)
    with open('train_target.pickle', 'rb') as data:
          train_target = pickle.load(data)
    with open('validate_target.pickle', 'rb') as data:
          validate_target = pickle.load(data)
    with open('test_target.pickle', 'rb') as data:
         test_target = pickle.load(data)
    return torch.tensor(train_input,dtype=torch.float32), torch.tensor(validate_input,dtype=torch.float32), torch.tensor(test_input,dtype=torch.float32), torch.tensor(train_target,dtype=torch.float32), torch.tensor(validate_target,dtype=torch.float32), torch.tensor(test_target,dtype=torch.float32)

In [33]:
train_input,validate_input,test_input,train_target,validate_target,test_target=load_data()
print("Train shape: " ,train_target.shape)

Train shape:  torch.Size([5099, 8, 2])


In [170]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, x, y):
        'Initialization'
        self.x = x
        self.y = y

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.x)

  def __getitem__(self, index):
        # Load data and get label
        X = self.x[index]
        y = self.y[index]

        return X, y

In [173]:
train_dataset= Dataset(train_input, train_target)
test_dataset= Dataset(test_input, test_target)

In [34]:
class LinearNetwork(nn.Module):
    def __init__(self):
        super(LinearNetwork, self).__init__()
        self.output_shape = (8,2)
        self.flatten = nn.Flatten()
        self.unflatten = nn.Unflatten(1,self.output_shape)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8*2, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 8*2),
        )

    def forward(self, x):
        x = self.flatten(x)
        # print(x[0])
        logits = self.linear_relu_stack(x)
        out = self.unflatten(logits)
        # print(out[0])
        return out

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [36]:
model = LinearNetwork().to(device)
print(model)

LinearNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (unflatten): Unflatten(dim=1, unflattened_size=(8, 2))
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=16, bias=True)
  )
)


In [37]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters())

In [85]:
train_input. requires_grad = True
train_input. requires_grad 

True

In [39]:
y_pred = model(train_input)
y_pred.shape
print (y_pred[0])
sq = torch.flatten(y_pred,1)
sq2 = torch.flatten(train_input,1)
print("///////////")
print(sq[0])
print(sq2[0])
# temp = nn.Flatten(train_input)
# train_input.shape,temp.shape,y_pred[0]


tensor([[-1.6634,  0.5903],
        [ 0.4768, -0.8592],
        [ 0.0575, -0.2041],
        [ 0.4909, -1.4022],
        [-0.4778,  1.3517],
        [ 0.2441, -0.9381],
        [-1.7007,  0.1728],
        [-0.5561, -0.0388]], grad_fn=<SelectBackward0>)
///////////
tensor([-1.6634,  0.5903,  0.4768, -0.8592,  0.0575, -0.2041,  0.4909, -1.4022,
        -0.4778,  1.3517,  0.2441, -0.9381, -1.7007,  0.1728, -0.5561, -0.0388],
       grad_fn=<SelectBackward0>)
tensor([14.9352,  5.3071, 14.4947,  5.3293, 14.0542,  5.3515, 13.6137,  5.3737,
        13.1726,  5.3937, 12.7264,  5.3937, 12.2802,  5.3937, 11.8340,  5.3937],
       grad_fn=<SelectBackward0>)


In [40]:
def train():
    for epoch in range(3000):
        y_pred = model(train_input) # make a prediction
        loss = loss_fn(y_pred,train_target) # compute the loss
        optimizer.zero_grad() # prepare the optimizer
        loss.backward() # compute the contribution of each parameter to the loss
        optimizer.step() # modify the parameters

        if epoch % 300 == 0:
            print(epoch, loss.data)

train()

0 tensor(47.2369)
300 tensor(0.6566)
600 tensor(0.7877)
900 tensor(0.6171)
1200 tensor(0.9639)
1500 tensor(0.7391)
1800 tensor(2.4894)
2100 tensor(0.8386)
2400 tensor(1.0475)
2700 tensor(0.4876)


In [58]:
def show_error_double(prediction_train,prediction_test,train_target,test_target):
                # # Show error rate
        print("AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Train: ",prediction_displacement_double(train_target))
        print("AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Test: ",prediction_displacement_double(test_target))
             
        #average_displacement_error
        print("ADE ERROR RATE TEST: ", ADE_double_coordinates(prediction_test,test_target))
        #average_displacement_error
        print("ADE ERROR RATE TRAIN: ", ADE_double_coordinates(prediction_train,train_target))
        print("//////////////////////////////////////////")
        #Final_displacement_error
        print("FDE ERROR RATE TEST: ", FDE_double_coordinates(prediction_test,test_target))
        print("FDE ERROR RATE TRAIN: ", FDE_double_coordinates(prediction_train,train_target))


In [307]:
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_features, hidden_units,out_features=16,isseq=True):
        super().__init__()
        self.seq = isseq
        self.num_features = num_features  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=out_features)

    def forward(self, x):
        batch_size = x.shape[0]
        # print("batch_size",batch_size)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        if(self.seq):
            out = self.linear(hn[0]).reshape(batch_size,8,2) # First dim of Hn is num_layers, which is set to 1 above.
        else:
            out = self.linear(hn[0])
        return out

In [308]:
learning_rate = 5e-5
num_hidden_units = 16
hidden_size = 12
n_features = 2
num_layers = 1
batch_size = 5

LSTM_model = ShallowRegressionLSTM(num_features=n_features, hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(LSTM_model.parameters(), lr=learning_rate)

In [309]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size)

In [310]:
def train_model(data_loader, model, loss_function, optimizer,epochs=10):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for i in range (epochs):
        total_loss = 0
        for X, y in data_loader:
            output = model(X)
            # print(output.shape)
            # print(y.shape)
            loss = loss_function(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / num_batches
        if (i+1)%5 == 0:
            print(f"Epoch {i+1} train loss: {avg_loss}")

In [311]:
train_model(train_loader, LSTM_model, loss_function, optimizer,epochs=50)

Epoch 5 train loss: 12.246922252341813
Epoch 10 train loss: 5.72934009675667
Epoch 15 train loss: 4.608505003731333
Epoch 20 train loss: 3.767580728076727
Epoch 25 train loss: 3.0283553304497666
Epoch 30 train loss: 2.4091508976932543
Epoch 35 train loss: 1.8859620055530733
Epoch 40 train loss: 1.4727450577405226
Epoch 45 train loss: 1.1456869817023878
Epoch 50 train loss: 0.8708728728605909


In [312]:
with torch.no_grad():
    test_pred = LSTM_model(test_input)
    train_pred = LSTM_model(train_input)
    loss = loss_fn(test_target, test_pred)
    print(loss.data)

tensor(1.3789)


In [313]:
show_error_double(train_pred,test_pred,train_target,test_target)

AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Train:  1.7491152588806866
AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Test:  2.7941778349259665
ADE ERROR RATE TEST:  1.333707828970117
ADE ERROR RATE TRAIN:  0.9344492040147702
//////////////////////////////////////////
FDE ERROR RATE TEST:  2.2891640075773427
FDE ERROR RATE TRAIN:  1.599989753412669


In [314]:
# Predicting only one point in the future
train_target_one = train_target[:,0,:]
test_target_one = test_target[:,0,:]
train_dataset_one= Dataset(train_input, train_target_one)
test_dataset_one= Dataset(test_input, test_target_one)
train_loader_one = torch.utils.data.DataLoader(train_dataset_one, batch_size)
test_loader_one = torch.utils.data.DataLoader(test_dataset_one, batch_size)

In [315]:
learning_rate = 5e-5
num_hidden_units = 16
hidden_size = 12
n_features = 2
num_layers = 1
batch_size = 5

LSTM_model2 = ShallowRegressionLSTM(num_features=n_features, hidden_units=num_hidden_units,out_features=2,isseq=False)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(LSTM_model2.parameters(), lr=learning_rate)

In [316]:
train_model(train_loader_one, LSTM_model2, loss_function, optimizer,epochs=50)

Epoch 5 train loss: 7.6770257825506665
Epoch 10 train loss: 3.028978386417479
Epoch 15 train loss: 1.7102254427592247
Epoch 20 train loss: 1.043713672121949
Epoch 25 train loss: 0.6145822825751869
Epoch 30 train loss: 0.3691840190514317
Epoch 35 train loss: 0.233049951416886
Epoch 40 train loss: 0.15658085884426298
Epoch 45 train loss: 0.11153401528309212
Epoch 50 train loss: 0.08315793621456988


In [317]:
with torch.no_grad():
    test_pred_one= LSTM_model2(test_input)
    train_pred_one = LSTM_model2(train_input)
    loss_train = loss_fn(train_target_one, train_pred_one)
    loss_test = loss_fn(test_target_one, test_pred_one)
    print(f'loss_train: {loss_train.data} \nloss_test: {loss_test.data}')

loss_train: 0.08635770529508591 
loss_test: 0.1889965832233429


In [318]:
def ADE(pred,truth): 
    counter=0
    sum=0
    pred = pd.DataFrame(pred.cpu().numpy())
    truth = pd.DataFrame(truth.cpu().numpy())
    print(pred.shape)
    for i in range(len(pred)):
        half=int(len(pred[i])/2)
        for j in range (half):

            a = np.array((pred[i][j] , pred[i][half+j]))
            b = np.array((truth.iloc[i][j] , truth.iloc[i][half+j]))

            dist = np.linalg.norm(a-b)
            sum+=dist
            counter+=1
            #print("Distance between",a," and ",b," is: ",dist)

    return (sum/counter)
def FDE(pred,truth): 
    counter=0
    sum=0
    pred = pred.cpu().numpy()
    truth = truth.cpu().numpy()

    for ai,bi in zip(pred,truth):
        # print(ai,bi)
        dist = np.linalg.norm(ai-bi)
        sum+=dist
        counter+=1
    return (sum/counter)


def show_error_single(prediction_train,prediction_test,train_target,test_target):
                # # Show error rate
        print("AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Train: ",prediction_displacement(train_target))
        print("AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Test: ",prediction_displacement(test_target))
        print("////////////////////////////////////////////////")
        #average_displacement_error
        print("ADE ERROR RATE TEST: ", FDE(prediction_test,test_target))
        #average_displacement_error
        print("ADE ERROR RATE TRAIN: ", FDE(prediction_train,train_target))
        print("//////////////////////////////////////////")
        #Final_displacement_error
        print("FDE ERROR RATE TEST: ", FDE(prediction_test,test_target))
        print("FDE ERROR RATE TRAIN: ", FDE(prediction_train,train_target))

In [319]:
show_error_single(train_pred_one,test_pred_one,train_target_one,test_target_one)

AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Train:  0.0
AVERAGE DISTANCE BETWEEN FIRST AND LAST POINT Test:  0.0
////////////////////////////////////////////////
ADE ERROR RATE TEST:  0.4339348352825256
ADE ERROR RATE TRAIN:  0.31064707524822427
//////////////////////////////////////////
FDE ERROR RATE TEST:  0.4339348352825256
FDE ERROR RATE TRAIN:  0.31064707524822427
